In [19]:
# Importamos data de titulos de noticias de los últimos 15 años obtenidos de Kaggle
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', encoding = "latin-1", error_bad_lines=False);
data_text = data[['headline_text']]

#Agrega una columna adicional como index basado en los indexes de la misma estructura
data_text['index'] = data_text.index
documents = data_text


print(len(documents))
print(documents[:5])



24909
                                       headline_text  index
0  Analista Funcional Senior TI Adexus Peru Lima ...      0
1  Practicante Supervision Bancaria Dpto c Superi...      1
2  Analista Programador Java Semi Senior TI Adexu...      2
3  Analista Programador Java Junior TI Adexus Per...      3
4  Analista Programador .Net Senior TI Adexus Per...      4


In [20]:
# Preprocesamiento de datos

#Ejecutaremos los siguientes pasos:

# 1. Tokenization: Parte el texto en sentencias y las sentencias en palabras. Las palabras se ponen en minuscula
#                  y se remueve la puntuación
# 2. Palabras que tienen menos de 3 caracteres son removidos.
# 3. Se eliminan todas las palabras de parada
# 4. Las palabras son lematizadas: Las palabras en tercera persona son cambiadas a primera persona 
#                                  y los verbos en pasado y futuro son cambiado a presente
# 5. Las palabras se derivan: Las palabras se reducen en su forma raiz

# Loading gensim and nltk libraries

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vpc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
# Se escribe una funcion para ejecutar la lematización y preprocesamiento en el conjunto de datos

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [23]:
# seleccionamos un documento para visualizar luego del preprocesamiento
stemmer = PorterStemmer()

doc_sample = documents[documents['index'] == 10310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Si', 'estás', 'en', 'el', 'I', 'Seminario', 'de', 'Buenas', 'Prácticas', 'Laborales,', 'recuerda', 'que', 'utilizando', 'el', 'hashtag', '#SeminarioAptitus', 'pued\x85', 'https://t.co/wf22t8Jdwf']


 tokenized and lemmatized document: 
['está', 'seminario', 'buena', 'práctica', 'laboral', 'recuerda', 'utilizando', 'hashtag', 'pu', 'http', 'jdwf']


In [24]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

print(processed_docs[:5])

0    [analista, funcion, senior, adexu, peru, lima,...
1    [practicant, supervis, bancaria, dpto, banca, ...
2    [analista, programador, java, semi, senior, ad...
3    [analista, programador, java, junior, adexu, p...
4    [analista, programador, senior, adexu, peru, l...
Name: headline_text, dtype: object


In [25]:
# Bag of Words on the Data set
# Crear un diccionaro basado en 'processed_docs' que contiene 
# el número de veces que una palabra aparece en los datos de entrenamiento
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 adexu
1 analista
2 funcion
3 http
4 lima
5 peru
6 senior
7 yryujjmi
8 banca
9 bancaria
10 dpto
11 practicant
12 qslyklox
13 supervis
14 java
15 programador
16 semi
17 zrim
18 junior
19 mfej
20 uprm


In [26]:
# Gensim filter_extremes

#  Filtra los tokens que aparecen en
# 1. menos de 15 documentos (número absoluto) o 
# 2. más que 0.5 documentos (fracción del tamaño del cuerpo total, no es número absoluto)
# 3. Después de los dos pasos anteriores, mantenga solo los primeros 100000 tokens más frecuentes.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [27]:
# Gensim doc2bow
# Para cada documento creamos un diccionario que reporte cuantas palabras
# and cuantas veces estas palabras aparecen. Guarde esto en 'bow_corpus',
# entonces checkee el documento seleccionado previamente.

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[1310]

[(0, 1), (339, 1)]

In [28]:
# Previsualizar Bag of Words para nuestra muestra documentos procesados
bow_doc_1310 = bow_corpus[1310]
for i in range(len(bow_doc_1310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1310[i][0], 
                                               dictionary[bow_doc_1310[i][0]], 
bow_doc_1310[i][1]))

Word 0 ("analista") appears 1 time.
Word 339 ("trabajando") appears 1 time.


In [29]:
# TF-IDF
# Crear el objeto de modelo tf-idf usando models.TfidModel en 'bow_corpus'
# y guardarlo en 'tdidf', luego aplicar tranformación al cuerpo entero y
# llamarlo 'corpus_tfidf'. Finalmente previsualizamos los scores TF-IDF
# para nuestro primer documento

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.34707701501073235),
 (1, 0.5948847080332527),
 (2, 0.26954550707210595),
 (3, 0.3934193540003902),
 (4, 0.5460917151125703)]


In [30]:
# Running LDA using Bag of Words
# Entrenar nuestro modelo lda usando gensim.models.LdaMulticore y 
# guardarlo en 'lda_model'

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [31]:
# Para cada tópico, exploraremos las palabras que aparecen en ese tópico y su peso relativo.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
# Puedes distinguir los diferentes tópicos usando las palabras en cada tópico y sus pesos correspondientes?

Topic: 0 
Words: 0.079*"overal" + 0.041*"perú" + 0.035*"está" + 0.030*"lima" + 0.028*"mbtska" + 0.027*"empresa" + 0.027*"transna" + 0.026*"buscando" + 0.024*"empleo" + 0.023*"búsqueda"
Topic: 1 
Words: 0.060*"nuestra" + 0.046*"atención" + 0.044*"oportunidad" + 0.038*"postula" + 0.037*"esta" + 0.033*"para" + 0.031*"asistent" + 0.029*"mbtska" + 0.029*"ingresa" + 0.023*"client"
Topic: 2 
Words: 0.064*"para" + 0.031*"empleo" + 0.029*"laboral" + 0.025*"trabajo" + 0.021*"oferta" + 0.019*"mejor" + 0.019*"tien" + 0.017*"week" + 0.016*"labor" + 0.015*"overal"
Topic: 3 
Words: 0.158*"overal" + 0.063*"corporativo" + 0.052*"servicio" + 0.041*"ejecutivo" + 0.039*"desarrollo" + 0.028*"impulsamo" + 0.025*"busi" + 0.023*"impulsadora" + 0.018*"mbtska" + 0.017*"vshyfm"
Topic: 4 
Words: 0.054*"ingresa" + 0.049*"búsqueda" + 0.048*"encontramo" + 0.047*"mbtska" + 0.034*"asistent" + 0.032*"practicant" + 0.024*"profesion" + 0.020*"lima" + 0.019*"nuestro" + 0.019*"portal"
Topic: 5 
Words: 0.072*"para" + 0.043*

In [32]:
# Running LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))
    
# Nuevamente, puedes distinguir los diferentes tópicos usando las palabras en cada tópico y sus pesos correspondientes?

Topic: 0 Word: 0.020*"perú" + 0.019*"portaltrabajo" + 0.015*"empleo" + 0.014*"transna" + 0.013*"para" + 0.013*"empresa" + 0.010*"lima" + 0.009*"búsqueda" + 0.009*"postula" + 0.008*"limpieza"
Topic: 1 Word: 0.018*"overal" + 0.016*"para" + 0.016*"corporativo" + 0.016*"oportunidad" + 0.013*"comienza" + 0.013*"promocion" + 0.013*"área" + 0.013*"trabajo" + 0.012*"portaltrabajo" + 0.012*"administrativa"
Topic: 2 Word: 0.020*"perú" + 0.018*"portaltrabajo" + 0.018*"asesor" + 0.016*"comerci" + 0.015*"experiencia" + 0.015*"empleo" + 0.014*"transna" + 0.013*"empresa" + 0.011*"teleoperador" + 0.011*"impulsamo"
Topic: 3 Word: 0.014*"humano" + 0.012*"facebook" + 0.011*"para" + 0.011*"nuestra" + 0.010*"analista" + 0.010*"overal" + 0.009*"trabajo" + 0.009*"recurso" + 0.008*"profesion" + 0.007*"oscar"
Topic: 4 Word: 0.028*"portal" + 0.026*"completo" + 0.025*"perfil" + 0.023*"postular" + 0.021*"pued" + 0.017*"vshyfm" + 0.015*"overal" + 0.015*"ingresando" + 0.014*"visita" + 0.013*"empleo"
Topic: 5 Word: 

In [37]:
# Evaluación del desempeño clasificando un documento de muestra usando 
# el modelo LDA Bag of Words
# Checkeare donde nuestro documento de texto sería clasificado

processed_docs[2]

['analista',
 'programador',
 'java',
 'semi',
 'senior',
 'adexu',
 'peru',
 'lima',
 'http',
 'zrim']

In [38]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    
# Nuestro documento de prueba tiene la mayor probabilidad de ser parte del tema que 
# nuestro modelo asignó, que es la clasificación precisa.


Score: 0.8714066743850708	 
Topic: 0.064*"empresa" + 0.049*"empleo" + 0.039*"perú" + 0.035*"trabajo" + 0.030*"entrevista" + 0.027*"perfil" + 0.026*"completo" + 0.025*"líder" + 0.023*"transna" + 0.023*"venta"

Score: 0.01429153885692358	 
Topic: 0.060*"nuestra" + 0.046*"atención" + 0.044*"oportunidad" + 0.038*"postula" + 0.037*"esta" + 0.033*"para" + 0.031*"asistent" + 0.029*"mbtska" + 0.029*"ingresa" + 0.023*"client"

Score: 0.01429047528654337	 
Topic: 0.054*"ingresa" + 0.049*"búsqueda" + 0.048*"encontramo" + 0.047*"mbtska" + 0.034*"asistent" + 0.032*"practicant" + 0.024*"profesion" + 0.020*"lima" + 0.019*"nuestro" + 0.019*"portal"

Score: 0.01428951881825924	 
Topic: 0.158*"overal" + 0.063*"corporativo" + 0.052*"servicio" + 0.041*"ejecutivo" + 0.039*"desarrollo" + 0.028*"impulsamo" + 0.025*"busi" + 0.023*"impulsadora" + 0.018*"mbtska" + 0.017*"vshyfm"

Score: 0.014288264326751232	 
Topic: 0.079*"overal" + 0.041*"perú" + 0.035*"está" + 0.030*"lima" + 0.028*"mbtska" + 0.027*"empresa" 

In [40]:
# Evaluación del desempeño clasificando un documento de muestra utilizando el modelo LDA TF-IDF.
for index, score in sorted(lda_model_tfidf[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    
# Nuestro documento de prueba tiene la mayor probabilidad de ser parte del tema que nuestro 
# modelo asignó, que es la clasificación precisa.


Score: 0.8714082837104797	 
Topic: 0.016*"perú" + 0.016*"empresa" + 0.016*"client" + 0.016*"programador" + 0.015*"nuestro" + 0.014*"jefe" + 0.012*"empleo" + 0.012*"venta" + 0.011*"servicio" + 0.010*"analista"

Score: 0.014290133491158485	 
Topic: 0.063*"perú" + 0.040*"transna" + 0.036*"empleo" + 0.035*"empresa" + 0.020*"lima" + 0.016*"semana" + 0.015*"venta" + 0.015*"buen" + 0.011*"vendedor" + 0.010*"inicio"

Score: 0.014288833364844322	 
Topic: 0.014*"humano" + 0.012*"facebook" + 0.011*"para" + 0.011*"nuestra" + 0.010*"analista" + 0.010*"overal" + 0.009*"trabajo" + 0.009*"recurso" + 0.008*"profesion" + 0.007*"oscar"

Score: 0.014288203790783882	 
Topic: 0.020*"perú" + 0.019*"portaltrabajo" + 0.015*"empleo" + 0.014*"transna" + 0.013*"para" + 0.013*"empresa" + 0.010*"lima" + 0.009*"búsqueda" + 0.009*"postula" + 0.008*"limpieza"

Score: 0.014287931844592094	 
Topic: 0.028*"portal" + 0.026*"completo" + 0.025*"perfil" + 0.023*"postular" + 0.021*"pued" + 0.017*"vshyfm" + 0.015*"overal" + 0

In [36]:
# Testing model on unseen document
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3666684627532959	 Topic: 0.064*"para" + 0.031*"empleo" + 0.029*"laboral" + 0.025*"trabajo" + 0.021*"oferta"
Score: 0.3666570484638214	 Topic: 0.034*"para" + 0.018*"overal" + 0.015*"pued" + 0.014*"trabajo" + 0.014*"ingresa"
Score: 0.03334025293588638	 Topic: 0.158*"overal" + 0.063*"corporativo" + 0.052*"servicio" + 0.041*"ejecutivo" + 0.039*"desarrollo"
Score: 0.03333413600921631	 Topic: 0.072*"para" + 0.043*"teleoperador" + 0.019*"empresa" + 0.018*"portaltrabajo" + 0.016*"montacarguista"
Score: 0.03333338722586632	 Topic: 0.079*"overal" + 0.041*"perú" + 0.035*"está" + 0.030*"lima" + 0.028*"mbtska"
Score: 0.03333333507180214	 Topic: 0.060*"nuestra" + 0.046*"atención" + 0.044*"oportunidad" + 0.038*"postula" + 0.037*"esta"
Score: 0.03333333507180214	 Topic: 0.054*"ingresa" + 0.049*"búsqueda" + 0.048*"encontramo" + 0.047*"mbtska" + 0.034*"asistent"
Score: 0.03333333507180214	 Topic: 0.058*"empresa" + 0.042*"venta" + 0.030*"promotor" + 0.027*"portaltrabajo" + 0.027*"nuestro"
Score: